# Assignment 1

In [1]:
import numpy as np
import pennylane as qml

## 1.2

In [2]:
def prepare_state(theta, phi):
    """Quantum function to prepare states of the form
        cos(\theta/2) |0> + e^(i\phi) sin(\theta/2) |1>.

    In principle, this function would be hidden, with no knowledge of theta/phi,
    in order to prepare a truly random state. It is provided here just for
    expository purposes.
    """
    qml.RY(theta, wires=0)
    qml.RZ(phi, wires=0)

In [20]:
def extract_bloch_vector(theta, phi):
    """Given a set of angular parameters representing the quantum state
        cos(\theta/2) |0> + e^(i\phi) sin(\theta/2) |1>,
    compute the Bloch vector associated to this state.

    The Bloch vector has three real-valued elements representing the position of
    the quantum state in the 3-dimensional space of the Bloch sphere. It can
    be computed by measuring a cleverly-chosen set of expectation values.

    Args:
        theta (float): Angular parameter of the states.
        phi (float): Phase parameter of the state.

    Returns:
        bloch_vector (array[float, float, float]): A NumPy array representing the
        3-element Bloch vector.
    """
    # Note here that we use an analytic device so that we get exact results
    dev = qml.device("default.qubit", wires=1)

    # YOUR CODE HERE
    # Use theta, phi *only* as arguments to prepare_state
    @qml.qnode(dev)
    def circuit():
        prepare_state(theta, phi)
        return np.array([qml.expval(qml.PauliX(0)), qml.expval(qml.PauliY(0)), qml.expval(qml.PauliZ(0))])
    
    return circuit()

In [22]:
print(extract_bloch_vector(np.pi, 0))

[ 0.  0. -1.]


## 1.3

In [78]:
def part_a():
    """Write a quantum circuit that computes the Boolean function
        f(a, b, c, d) = ab + cd
    where the + here represents the XOR operation.

    The first four qubits of your circuit should correspond to the
    input variables, a, b, c, d, in that order. The last qubit of your
    circuit should correspond to the output. If you use an auxiliary wires
    in your circuit, these should be uncomputed, like so:
          _______
     a ---|  C  |--- a
     b ---|  I  |--- b
     c ---|  R  |--- c
     d ---|  C  |--- d
     0 ---|     |--- 0 # Optional aux. wires
     0 ---|     |--- 0
          ...
     0 ---=======--- ab + cd

    """

    # You can change this to as many wires as you need
    num_wires = 7

    # Do not change the number of shots on the device
    dev = qml.device("default.qubit", wires=num_wires, shots=1)

    @qml.qnode(dev)
    def my_circuit(a, b, c, d):
        """ Args:
                a, b, c, d (int): The input variables, either 0 or 1.

            Returns:
                array[int]: A single sample of the output of your circuit.
        """
        
        # YOUR CODE HERE
        # Apply some quantum gates
        # Don't forget to initialize the first 4 wires based on a, b, c, d
        if a:
            qml.PauliX(wires=0)
        if b:
            qml.PauliX(wires=1)
        if c:
            qml.PauliX(wires=2)
        if d:
            qml.PauliX(wires=3)
            
        qml.Toffoli(wires=[0,1,4])
        qml.Toffoli(wires=[2,3,6])
        qml.CNOT(wires=[4,6])
        qml.Toffoli(wires=[0,1,4])

        return qml.sample()

    return my_circuit


In [79]:
print(part_a()(1,1,1,0))

[1 1 1 0 0 0 1]


In [150]:
def part_b():
    """Write a quantum circuit that computes the Boolean function
        f(a, b, c) = NOT(a) * b * c
    where the * here represents the AND operation.

    The first three qubits of your circuit should correspond to the
    input variables, a, b, c, in that order. The last qubit of your
    circuit should correspond to the output. If you use an auxiliary wires
    in your circuit, these should be uncomputed.

          _______
     a ---|  C  |--- a
     b ---|  I  |--- b
     c ---|  R  |--- c
     0 ---|  C  |--- 0 # Optional aux. wires
     0 ---|     |--- 0
            ...
     0 ---=======--- NOT(a) * b * c

    """

    # You can change this to as many wires as you need
    num_wires = 6

    # Do not change the number of shots on the device
    dev = qml.device("default.qubit", wires=num_wires, shots=1)

    @qml.qnode(dev)
    def my_circuit(a, b, c):
        """ Args:
                a, b, c (int): The input variables, either 0 or 1.

            Returns:
                array[int]: A single sample of the output of your circuit.
        """
        
        # YOUR CODE HERE
        # Apply some quantum gates
        if a:
            qml.PauliX(wires=0)
        if b:
            qml.PauliX(wires=1)
        if c:
            qml.PauliX(wires=2)
            
        qml.PauliX(wires=3)
        qml.CNOT(wires=[0,3])
        qml.Toffoli(wires=[1,2,4])
        qml.Toffoli(wires=[3,4,5])
        qml.Toffoli(wires=[1,2,4])
        qml.CNOT(wires=[0,3])
        qml.PauliX(wires=3)
        
        return qml.sample(wires=[0,1,2,5])

    return my_circuit

In [151]:
print(part_b()(0,1,1))

[0 1 1 1 1 1]


In [158]:

def part_c():
    """Write a quantum circuit that computes the Boolean function
        f(a, b, c) = a + a * NOT(b) + NOT(a * b * c)
    where the * here represents the AND operation, and + the XOR.

    The first three qubits of your circuit should correspond to the
    input variables, a, b, c, in that order. The last qubit of your
    circuit should correspond to the output. If you use an auxiliary wires
    in your circuit, these should be uncomputed.
          _______
     a ---|  C  |--- a
     b ---|  I  |--- b
     c ---|  R  |--- c
     0 ---|  C  |--- 0 # Optional aux. wires
     0 ---|     | 0
            ...
     0 ---=======---- a + a * NOT(b) + NOT(a * b * c)

    """

    # You can change this to as many wires as you need
    num_wires = 6

    # Do not change the number of shots on the device
    dev = qml.device("default.qubit", wires=num_wires, shots=1)

    @qml.qnode(dev)
    def my_circuit(a, b, c):
        """ Args:
                a, b, c (int): The input variables, either 0 or 1.

            Returns:
                array[int]: A single sample of the output of your circuit.
        """
        
        # YOUR CODE HERE
        # Apply some quantum gates
        if a:
            qml.PauliX(wires=0)
        if b:
            qml.PauliX(wires=1)
        if c:
            qml.PauliX(wires=2)
            
        qml.PauliX(wires=1)
        qml.Toffoli(wires=[0,1,3])
        qml.PauliX(wires=1)
        qml.CNOT(wires=[0,3]) # a + a*NOT(b)
        
        qml.Toffoli(wires=[0,1,5])
        qml.Toffoli(wires=[2,5,4])
        qml.Toffoli(wires=[0,1,5])
        qml.PauliX(wires=4) # NOT(a*b*c)
        
        qml.CNOT(wires=[3,4])
        qml.CNOT(wires=[0,3])
        
        return qml.sample(wires=[0,1,2,3,4])

    return my_circuit

In [168]:
#f(a, b, c) = a + a * NOT(b) + NOT(a * b * c)
a,b,c = 1,1,0
print(part_c()(a,b,c))
x = a and not b
y = not (a and b and c)
xor = (a and not x) or (not a and x)
xor = xor != y
print(x,y,xor)

[1 1 0 0 0]
False True False
